In [1]:
import pandas as pd
import pydeck as pdk

In [2]:
from odp.client import OdpClient # The SDK
from odp.dto import ResourceDto # Resource Data Transfer Object
from odp.client.dto.table_spec import TableSpec # Table Specification
from odp.client.exc import OdpResourceNotFoundError

In [3]:
client = OdpClient()

In [ ]:
## Import PGS data from ODP and filter for humpback whales

In [4]:
dataset = client.catalog.get(("c571f34a-490e-4a23-97ef-81c9efa41558"))
dataset.metadata.display_name

OdpResourceNotFoundError: Resource not found: c571f34a-490e-4a23-97ef-81c9efa41558

In [ ]:
filter_query = {
  "#EQUALS": [
    "$Species",
    "Megaptera novaeangliae"
  ]
}

In [ ]:
PGSdata = client.tabular.select_as_dataframe(dataset, filter_query=filter_query)
PGSdata["Latitude"] = PGSdata["geometry"].apply(lambda x: x["coordinates"][1] if x else None)
PGSdata["Longitude"] = PGSdata["geometry"].apply(lambda x: x["coordinates"][0] if x else None)
PGSdata.head()

## Load the corresponding data from OBIS from a csv file

In [ ]:
OBISdata = pd.read_csv('data/OBIS_humpback_whales.csv')

In [ ]:
OBISdata.rename(columns={"decimallatitude": "Latitude"}, inplace=True)
OBISdata.rename(columns={"decimallongitude": "Longitude"}, inplace=True)
OBISdata.head()

## Create a Plot of PGS and OBIS data on Humback Whales
- PGS is orange
- OBIS is Yellow

In [ ]:
def create_pydeck_scatter_plot(PGSdata, OBISdata, lon_col, lat_col):
    """
    Creates a Pydeck scatter plot based on longitude and latitude columns for two dataframes.

    Args:
        PGSdata (pd.DataFrame): Input DataFrame for PGS data.
        OBISdata (pd.DataFrame): Input DataFrame for OBIS data.
        lon_col (str): Name of the longitude column.
        lat_col (str): Name of the latitude column.

    Returns:
        pdk.Deck: Pydeck scatter plot.
    """
    scatter_layer_pgs = pdk.Layer(
        "ScatterplotLayer",
        data=OBISdata,
        get_position=[lon_col, lat_col],
        get_radius=10000,
        get_fill_color=[255, 215, 14],  # Yellow color for OBIS
        pickable=True,
    )

    scatter_layer_obis = pdk.Layer(
        "ScatterplotLayer",
        data=PGSdata,
        get_position=[lon_col, lat_col],
        get_radius=10000,
        get_fill_color=[245, 61, 1],  # Orange color for PGS
        pickable=True,
    )

    view_state = pdk.ViewState(
        latitude=(PGSdata[lat_col].mean() + OBISdata[lat_col].mean()) / 2,
        longitude=(PGSdata[lon_col].mean() + OBISdata[lon_col].mean()) / 2,
        zoom=4,
    )

    r = pdk.Deck(
        layers=[scatter_layer_pgs, scatter_layer_obis],
        initial_view_state=view_state,
        api_keys={'mapbox':'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'},
        map_provider="mapbox",
        map_style="mapbox://styles/oceandatafoundation/clwg6xklg00an01pcgmeufjxq",
    )
    return r


In [ ]:
deck = create_pydeck_scatter_plot(PGSdata, OBISdata, lon_col="Longitude", lat_col="Latitude")


In [ ]:
# Save the map as a html file
deck.to_html('scatter_plot.html')